# Data Collection

## Import Packages

In [1]:
from bs4 import BeautifulSoup
import requests
import json
import time
import pandas as pd
import numpy as np
import threading

# pd.set_option("display.max_rows", None, "display.max_columns", None)

## Web Scrape Functions

Our first function will have us finding information on census.gov for given cities and storing this into several different lists. 

information to a dataframe.

In [2]:
df_dict = {'CITY': [], #city
           'STATE': [], #state
           'PA': [], #political affiliation
           'POP': [], #population
           'UNDER_5': [], #% of people under age 5
           'UNDER_18': [], #% of people under age 18
           '65_OR_OVER': [], #% of people 65 and older
           'FEMALE': [], #% female
          'WHITE': [], #% white only
          'BLACK': [], #% black only
          'AMERICAN_INDIAN': [], #% american indian or alaska native only
          'ASIAN': [], #% asian only
           'PACIFIC_ISLANDER': [], #% native hawaiian and other pacific islander only
          'MULTI_RACE': [], #% two+ races
          'HISPANIC': [], #% hispanic
          'VETERAN': [], #num of veterans
          'FOREIGN': [], #% foreign born
          'HOUSES': [], #% of owner occupied houses
          'HOUSE_VAL': [], #median value of owner occupied houses
          'RENT': [], #median gross rent
          'HOUSEHOLDS': [], #households
          'PPH': [], #people per household
          'YEAR_IN_HOUSE': [], #% of people living in same house for one year or more
          'OTHER_HOME_LANG': [], #% of people, 5+ years old, who speak a language other than english at home
          'COMPUTER': [], #% of households with computer
          'INTERNET': [], #% of households with internet
          'HIGH_SCHOOL': [], #% of people, 25+ years old, with high school diploma or higher 
          'BACH_DEGREE': [], #% of people, 25+ years old, with bachelor's degree or higher
          'DISABILITY_UNDER_65': [], #% of people with a disability under 65 years old
          'NO_INSURANCE': [], #% of people without health insurance, under 65 years old
          'LABOR': [], #% of population in civilian labor force, 16+ years old
          'FEM_LABOR': [], #% of female population in civilian labor force, 16+ years old
          'HEALTHCARE': [], #healthcare and social assistance revenue ($1,000)
          'SHIPMENTS': [], #manufacturers shipments ($1,000)
          'TRAVEL_TIME': [], #average travel time to work (min), 16+ years old
          'HOUSEHOLD_INCOME': [], #average household income
          'INCOME': [], #per capita income in last 12 months
          'POVERTY': [], #% of people in poverty
          'FIRMS': [], #all firms
          'MEN_FIRM': [], #men owned firms
          'FEM_FIRM': [], #female owned firms
          'MINOR_FIRM': [], #minority owned firms
          'NON_MINOR_FIRM': [], #nonminority owned firms
          'VET_FIRM': [], #veteran owned firms
          'NON_VET_FIRM': [], #non-veteran owned firms
          'POP_AREA': [], #population per square mile
          'AREA': [] #land area in square miles
          } 

In [3]:
def collect_city_data(city, state):
    banana = False
    """Collect census and political affiliation data for given city and append it to data dictionary."""
    #append city and state data to df dictionary
    # change city and state variables into format for census url
    census_city = city.lower().strip().replace(" ", "")
    census_state = state.lower().strip().replace(" ", "")
    # change city and state variables into format for political affiliation url
    aff_city = city.lower().replace(" ", "_")
    aff_state = state.lower().replace(" ", "_")
    try:
        #navigate to census page for this particular city
        census_url = f"https://www.census.gov/quickfacts/fact/table/{census_city}city{census_state}/PST045219"
        html_page = requests.get(census_url)
        html_tree = BeautifulSoup(html_page.content, 'html.parser')
        facts_container = html_tree.find('div', class_="qf-facttable")
        #if page times out, we try again
        while facts_container is None:
            time.sleep(1)
            census_url = f"https://www.census.gov/quickfacts/fact/table/{census_city}city{census_state}/PST045219"
            html_page = requests.get(census_url)
            html_tree = BeautifulSoup(html_page.content, 'html.parser')
            facts_container = html_tree.find('div', class_="qf-facttable")
        #pages not found on census website will redirect to USA facts page
        name_found = html_tree.find('div', class_="qf-geobox").find('span').get_text()
        #if this happens, raise an exception
        if name_found == 'United States':
            #print("Appending PA with NaN")
            df_dict['CITY'].append(city)
            df_dict['STATE'].append(state)
            df_dict['PA'].append("NaN")
            #print("I did it")
            raise ValueError("City Not Found")
            #print("You should never see me... john cena")
        #grab information for each row
        #population
        #if banana:
            #print("You should also never see me!")
        population_facts = facts_container.findAll('tbody')[1]
        population = population_facts.findAll('tr')[2].findAll('td')[1].attrs['data-value']
        #% of people under age 5
        age_and_sex_facts = facts_container.findAll('tbody')[2]
        person_under_5 = age_and_sex_facts.findAll('tr')[1].findAll('td')[1].attrs['data-value']
        #% of people under age 18
        person_under_18 = age_and_sex_facts.findAll('tr')[2].findAll('td')[1].attrs['data-value']
        #% of people 65 and older
        person_65_plus = age_and_sex_facts.findAll('tr')[3].findAll('td')[1].attrs['data-value']
        #% female
        female = age_and_sex_facts.findAll('tr')[4].findAll('td')[1].attrs['data-value']
        #% white only
        race_facts = facts_container.findAll('tbody')[3]
        white = race_facts.findAll('tr')[1].findAll('td')[1].attrs['data-value']
        #% black only
        black = race_facts.findAll('tr')[2].findAll('td')[1].attrs['data-value']
        #% american indian or alaska native only
        american_indian_or_alaska_native = race_facts.findAll('tr')[3].findAll('td')[1].attrs['data-value']
        #% asian only
        asian = race_facts.findAll('tr')[4].findAll('td')[1].attrs['data-value']
        #% native hawaiian and other pacific islander only
        native_hawaiian_and_other_pacific_islander = race_facts.findAll('tr')[5].findAll('td')[1].attrs['data-value']
        #% two+ races
        two_or_more_races = race_facts.findAll('tr')[6].findAll('td')[1].attrs['data-value']
        #% hispanic
        hispanic = race_facts.findAll('tr')[7].findAll('td')[1].attrs['data-value']
        #num of veterans
        pop_characteristic_facts = facts_container.findAll('tbody')[4]
        veteran = pop_characteristic_facts.findAll('tr')[1].findAll('td')[1].attrs['data-value']
        #% foreign born
        foreign_born = pop_characteristic_facts.findAll('tr')[2].findAll('td')[1].attrs['data-value']
        #% of owner occupied houses
        housing_facts = facts_container.findAll('tbody')[5]
        owner_occupied_houses = housing_facts.findAll('tr')[2].findAll('td')[1].attrs['data-value']
        #median value of owner occupied houses
        median_value_owner_occupied_houses = housing_facts.findAll('tr')[3].findAll('td')[1].attrs['data-value']
        #median gross rent
        median_rent = housing_facts.findAll('tr')[6].findAll('td')[1].attrs['data-value']
        #households
        family_facts = facts_container.findAll('tbody')[6]
        households = family_facts.findAll('tr')[1].findAll('td')[1].attrs['data-value']
        #people per household
        people_per_household = family_facts.findAll('tr')[2].findAll('td')[1].attrs['data-value']
        #% of people living in same house for one year or more
        same_house_oneplus_years = family_facts.findAll('tr')[3].findAll('td')[1].attrs['data-value']
        #% of people, 5+ years old, who speak a language other than english at home
        non_english_house = family_facts.findAll('tr')[4].findAll('td')[1].attrs['data-value']
        #% of households with computer
        computer_facts = facts_container.findAll('tbody')[7]
        household_w_computer = computer_facts.findAll('tr')[1].findAll('td')[1].attrs['data-value']
        #% of households with internet
        household_w_internet = computer_facts.findAll('tr')[2].findAll('td')[1].attrs['data-value']
        #% of people, 25+ years old, with high school diploma or higher
        education_facts = facts_container.findAll('tbody')[8]
        highschoolplus_25plus = education_facts.findAll('tr')[1].findAll('td')[1].attrs['data-value']
        #% of people, 25+ years old, with bachelor's degree or higher
        bachelorsplus_25plus = education_facts.findAll('tr')[2].findAll('td')[1].attrs['data-value']
        #% of people with a disability under 65 years old
        health_facts = facts_container.findAll('tbody')[9]
        disability_65under = health_facts.findAll('tr')[1].findAll('td')[1].attrs['data-value']
        #% of people without health insurance, under 65 years old
        no_healthinsurance_65under = health_facts.findAll('tr')[2].findAll('td')[1].attrs['data-value']
        #% of population in civilian labor force, 16+ years old
        economy_facts = facts_container.findAll('tbody')[10]
        total_laborforce_16plus = economy_facts.findAll('tr')[1].findAll('td')[1].attrs['data-value']
        #% of female population in civilian labor force, 16+ years old
        female_laborforce_16plus = economy_facts.findAll('tr')[2].findAll('td')[1].attrs['data-value']
        #healthcare and social assistance revenue ($1,000)
        healthcare_socialassistance_x1000 = economy_facts.findAll('tr')[4].findAll('td')[1].attrs['data-value']
        #manufacturers shipments ($1,000)
        manufacturer_shipments_x1000 = economy_facts.findAll('tr')[5].findAll('td')[1].attrs['data-value']
        #average travel time to work (min), 16+ years old
        transportation_facts = facts_container.findAll('tbody')[11]
        avg_travel_time_work_16plus = transportation_facts.findAll('tr')[1].findAll('td')[1].attrs['data-value']
        #average household income
        income_poverty_facts = facts_container.findAll('tbody')[12]
        median_household_income = income_poverty_facts.findAll('tr')[1].findAll('td')[1].attrs['data-value']
        #per capita income in last 12 months
        per_capita_income_12mo = income_poverty_facts.findAll('tr')[2].findAll('td')[1].attrs['data-value']
        #% of people in poverty
        people_in_poverty = income_poverty_facts.findAll('tr')[3].findAll('td')[1].attrs['data-value']
        #all firms
        business_facts = facts_container.findAll('tbody')[13]
        all_firms = business_facts.findAll('tr')[6].findAll('td')[1].attrs['data-value']
        #men owned firms
        men_owned_firms = business_facts.findAll('tr')[7].findAll('td')[1].attrs['data-value']
        #female owned firms
        women_owned_firms = business_facts.findAll('tr')[8].findAll('td')[1].attrs['data-value']
        #minority owned firms
        minority_owned_firms = business_facts.findAll('tr')[9].findAll('td')[1].attrs['data-value']
        #nonminority owned firms
        non_minority_owned_firms = business_facts.findAll('tr')[10].findAll('td')[1].attrs['data-value']
        #veteran owned firms
        veteran_owned_firms = business_facts.findAll('tr')[11].findAll('td')[1].attrs['data-value']
        #non-veteran owned firms
        non_veteran_owned_firms = business_facts.findAll('tr')[12].findAll('td')[1].attrs['data-value']
        #population per square mile
        geography_facts = facts_container.findAll('tbody')[14]
        population_per_sq_mile = geography_facts.findAll('tr')[1].findAll('td')[1].attrs['data-value']
        #land area in square miles
        land_in_sq_miles = geography_facts.findAll('tr')[2].findAll('td')[1].attrs['data-value']
        #navigate to bestplaces page for this particular city
        try:
            aff_url = f"https://www.bestplaces.net/voting/city/{aff_state}/{aff_city}"
            html_page = requests.get(aff_url)
            if "404" in html_page.url:
                raise requests.HTTPError()
            html_tree = BeautifulSoup(html_page.content, 'html.parser')
            container = html_tree.find_all('div', class_='col-md-12')
            while container is None:
                time.sleep(1)
                aff_url = f"https://www.bestplaces.net/voting/city/{aff_state}/{aff_city}"
                html_page = requests.get(aff_url)
                if "404" in html_page.url:
                    raise requests.HTTPError()
                html_tree = BeautifulSoup(html_page.content, 'html.parser')
                container = html_tree.find_all('div', class_='col-md-12')
            affiliation = container[1].find('b').text
            bad_chars = ['.', ','] 
            for i in bad_chars : 
                affiliation = affiliation.replace(i, '')
            affiliation = affiliation.split(' is ')[1].lower().split(' ')[1]
        except Exception as e:
            affiliation = "NaN"
            if e.__class__.__name__ == "HTTPError":
                print(f"{city} page not found in bestplaces.")
            else:
                print(f"An unexpected error occurred processing PA for {city}. Error: {e}")
        # append all collected data to the df dictionary
        df_dict['CITY'].append(city)
        df_dict['STATE'].append(state)
        df_dict['PA'].append(affiliation)
        df_dict['POP'].append(population)
        df_dict['UNDER_5'].append(person_under_5)
        df_dict['UNDER_18'].append(person_under_18)
        df_dict['65_OR_OVER'].append(person_65_plus)
        df_dict['FEMALE'].append(female)
        df_dict['WHITE'].append(white)
        df_dict['BLACK'].append(black)
        df_dict['AMERICAN_INDIAN'].append(american_indian_or_alaska_native)
        df_dict['ASIAN'].append(asian)
        df_dict['PACIFIC_ISLANDER'].append(native_hawaiian_and_other_pacific_islander)
        df_dict['MULTI_RACE'].append(two_or_more_races)
        df_dict['HISPANIC'].append(hispanic)
        df_dict['VETERAN'].append(veteran)
        df_dict['FOREIGN'].append(foreign_born)
        df_dict['HOUSES'].append(owner_occupied_houses)
        df_dict['HOUSE_VAL'].append(median_value_owner_occupied_houses)
        df_dict['RENT'].append(median_rent)
        df_dict['HOUSEHOLDS'].append(households)
        df_dict['PPH'].append(people_per_household)
        df_dict['YEAR_IN_HOUSE'].append(same_house_oneplus_years)
        df_dict['OTHER_HOME_LANG'].append(non_english_house)
        df_dict['COMPUTER'].append(household_w_computer)
        df_dict['INTERNET'].append(household_w_internet)
        df_dict['HIGH_SCHOOL'].append(highschoolplus_25plus)
        df_dict['BACH_DEGREE'].append(bachelorsplus_25plus)
        df_dict['DISABILITY_UNDER_65'].append(disability_65under)
        df_dict['NO_INSURANCE'].append(no_healthinsurance_65under)
        df_dict['LABOR'].append(total_laborforce_16plus)
        df_dict['FEM_LABOR'].append(female_laborforce_16plus)
        df_dict['HEALTHCARE'].append(healthcare_socialassistance_x1000)
        df_dict['SHIPMENTS'].append(manufacturer_shipments_x1000)
        df_dict['TRAVEL_TIME'].append(avg_travel_time_work_16plus)
        df_dict['HOUSEHOLD_INCOME'].append(median_household_income)
        df_dict['INCOME'].append(per_capita_income_12mo)
        df_dict['POVERTY'].append(people_in_poverty)
        df_dict['FIRMS'].append(all_firms)
        df_dict['MEN_FIRM'].append(men_owned_firms)
        df_dict['FEM_FIRM'].append(women_owned_firms)
        df_dict['MINOR_FIRM'].append(minority_owned_firms)
        df_dict['NON_MINOR_FIRM'].append(non_minority_owned_firms)
        df_dict['VET_FIRM'].append(veteran_owned_firms)
        df_dict['NON_VET_FIRM'].append(non_veteran_owned_firms)
        df_dict['POP_AREA'].append(population_per_sq_mile)
        df_dict['AREA'].append(land_in_sq_miles)    
    except Exception as err:
        for key, value in df_dict.items():
            if key == 'CITY' or key == 'STATE' or key == 'PA':
                continue
            else:
                df_dict[key].append("NaN")
        if err.__class__.__name__ == "HTTPError":
            print(f"{city} page not found in census.")
        else:
            print(f"An unexpected error occurred processing census for {city}. Error: {err}")
    return None

## Table of States and Cities in that State

In [4]:
top_cities = pd.read_csv('Top5000Population.csv')
states = pd.read_csv('data.csv')

In [5]:
top_cities.head()

,City,Code,Population
0,New York,NY,"8,363,710"
1,Los Angeles,CA,"3,833,995"
2,Chicago,IL,"2,853,114"
3,Houston,TX,"2,242,193"
4,Phoenix,AZ,"1,567,924"


In [6]:
top_cities['Population'] = top_cities['Population'].apply(lambda x: x.replace(",",""))

In [7]:
top_cities['Population'] = pd.to_numeric(top_cities['Population'])

In [8]:
top_cities = top_cities[top_cities['Population']>5000]

In [9]:
top_cities.tail()

,City,Code,Population
4562,Summerville,GA,5010
4563,Tipton,IN,5006
4564,Ayden,NC,5005
4565,Chelsea,MI,5005
4566,Thatcher,AZ,5002


In [10]:
states.head()

,State,Abbrev,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


,State,Abbrev,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


In [10]:
states['Cities'] = np.empty((len(states), 0)).tolist()

In [11]:
states.head()

,State,Abbrev,Code,Cities
0,Alabama,Ala.,AL,[]
1,Alaska,Alaska,AK,[]
2,Arizona,Ariz.,AZ,[]
3,Arkansas,Ark.,AR,[]
4,California,Calif.,CA,[]


In [12]:
for state_index, state_row in states.iterrows():
    for city_index, city_row in top_cities.iterrows():
        if city_row['Code'] == state_row['Code']:
            state_row['Cities'].append(city_row['City'])

In [13]:
states.head()

,State,Abbrev,Code,Cities
0,Alabama,Ala.,AL,"[Birmingham , Montgomery , Mobile , Huntsville..."
1,Alaska,Alaska,AK,"[Anchorage, Fairbanks , Juneau, Wasilla , Sitk..."
2,Arizona,Ariz.,AZ,"[Phoenix , Tucson , Mesa , Glendale , Chandler..."
3,Arkansas,Ark.,AR,"[Little Rock , Fort Smith , Fayetteville , Spr..."
4,California,Calif.,CA,"[Los Angeles , San Diego , San Jose , San Fran..."


## Table with Collected Data

In [14]:
#dictionary to track created threads
request_dict = {}
# bop = False

ccount = 4_565 # 4566 cities with a population > 5000
scount = 51 # 50 states
for state_index, state_row in states.iterrows():
#     if bop:
#         break
    state = state_row['State']
    for city in state_row['Cities']:
#         if ccount < 4000:
#             bop = True
#             break
        #create a thread to call collect_city_data with city and state as params. 
        request_dict[city] = threading.Thread(target=collect_city_data, args=(city, state))
        #start the newly made thread and move on to the next city
        request_dict[city].start()
        ccount -= 1
        print(f"{city} is done, {ccount} more to go!")
        time.sleep(0.1)
    scount -= 1
    print(f"{state} is done, {scount} more to go!")

Birmingham  is done, 4564 more to go!
Montgomery  is done, 4563 more to go!
Mobile  is done, 4562 more to go!
Huntsville  is done, 4561 more to go!
Tuscaloosa  is done, 4560 more to go!
Hoover  is done, 4559 more to go!
Dothan  is done, 4558 more to go!
Auburn  is done, 4557 more to go!
Decatur  is done, 4556 more to go!
Madison  is done, 4555 more to go!
Florence  is done, 4554 more to go!
Gadsden  is done, 4553 more to go!
Prattville  is done, 4552 more to go!
Phenix City  is done, 4551 more to go!
Vestavia Hills  is done, 4550 more to go!
Alabaster  is done, 4549 more to go!
Bessemer  is done, 4548 more to go!
Prichard  is done, 4547 more to go!
Opelika  is done, 4546 more to go!
Enterprise  is done, 4545 more to go!
Homewood  is done, 4544 more to go!
Athens  is done, 4543 more to go!
Anniston  is done, 4542 more to go!
Northport  is done, 4541 more to go!
Pelham  is done, 4540 more to go!
Mountain Brook  is done, 4539 more to go!
Oxford  is done, 4538 more to go!
Albertville  is d

Pine Bluff  is done, 4388 more to go!
Hot Springs  is done, 4387 more to go!
Bentonville  is done, 4386 more to go!
An unexpected error occurred processing census for Thatcher. Error: City Not Found
Jacksonville  is done, 4385 more to go!
Texarkana  is done, 4384 more to go!
Benton  is done, 4383 more to go!
Russellville  is done, 4382 more to go!
West Memphis  is done, 4381 more to go!
Paragould  is done, 4380 more to go!
Sherwood  is done, 4379 more to go!
Cabot  is done, 4378 more to go!
Van Buren  is done, 4377 more to go!
Searcy  is done, 4376 more to go!
El Dorado  is done, 4375 more to go!
Bella Vista is done, 4374 more to go!
Maumelle  is done, 4373 more to go!
Blytheville  is done, 4372 more to go!
Bryant  is done, 4371 more to go!
Siloam Springs  is done, 4370 more to go!
Forrest City  is done, 4369 more to go!
Harrison  is done, 4368 more to go!
Mountain Home  is done, 4367 more to go!
Marion  is done, 4366 more to go!
Helena-West Helena  is done, 4365 more to go!
Camden  is

Lake Elsinore  is done, 4177 more to go!
Palm Desert  is done, 4176 more to go!
Colton  is done, 4175 more to go!
Watsonville  is done, 4174 more to go!
Hanford  is done, 4173 more to go!
Gilroy  is done, 4172 more to go!
La Mirada  is done, 4171 more to go!
Yucaipa  is done, 4170 more to go!
Placentia  is done, 4169 more to go!
Rancho Santa Margarita  is done, 4168 more to go!
Brentwood  is done, 4167 more to go!
Glendora  is done, 4166 more to go!
San Ramon  is done, 4165 more to go!
Poway  is done, 4164 more to go!
Palm Springs  is done, 4163 more to go!
West Sacramento  is done, 4162 more to go!
Cypress  is done, 4161 more to go!
Covina  is done, 4160 more to go!
Azusa  is done, 4159 more to go!
Bell Gardens  is done, 4158 more to go!
Dublin  is done, 4157 more to go!
La Quinta  is done, 4156 more to go!
San Luis Obispo  is done, 4155 more to go!
Lincoln  is done, 4154 more to go!
Ceres  is done, 4153 more to go!
Newark  is done, 4152 more to go!
Aliso Viejo  is done, 4151 more to 

Anderson  is done, 3977 more to go!
Piedmont  is done, 3976 more to go!
An unexpected error occurred processing census for Hillsborough. Error: City Not Found
Mendota  is done, 3975 more to go!
Morro Bay  is done, 3974 more to go!
Newman  is done, 3973 more to go!
Shasta Lake  is done, 3972 more to go!
Farmersville  is done, 3971 more to go!
Placerville  is done, 3970 more to go!
Exeter  is done, 3969 more to go!
Sonoma  is done, 3968 more to go!
Capitola  is done, 3967 more to go!
Emeryville  is done, 3966 more to go!
Corte Madera is done, 3965 more to go!
Taft  is done, 3964 more to go!
Waterford  is done, 3963 more to go!
Tiburon is done, 3962 more to go!
Los Altos Hills is done, 3961 more to go!
Pismo Beach  is done, 3960 more to go!
Gonzales  is done, 3959 more to go!
Westlake Village  is done, 3958 more to go!
Live Oak  is done, 3957 more to go!
Cloverdale  is done, 3956 more to go!
Rolling Hills Estates  is done, 3955 more to go!
Crescent City  is done, 3954 more to go!
Rio Vist

Dover  is done, 3820 more to go!
An unexpected error occurred processing census for Naugatuck. Error: City Not Found
Newark  is done, 3819 more to go!
Middletown is done, 3818 more to go!
Smyrna is done, 3817 more to go!
Milford  is done, 3816 more to go!
Seaford  is done, 3815 more to go!
Elsmere is done, 3814 more to go!
Georgetown is done, 3813 more to go!
Delaware is done, 43 more to go!
Washington  is done, 3812 more to go!
An unexpected error occurred processing census for Middletown. Error: City Not Found
District of Columbia is done, 42 more to go!
Jacksonville  is done, 3811 more to go!
Miami  is done, 3810 more to go!
Tampa  is done, 3809 more to go!
An unexpected error occurred processing census for Elsmere. Error: City Not Found
An unexpected error occurred processing census for Georgetown. Error: City Not Found
St. Petersburg  is done, 3808 more to go!
Orlando  is done, 3807 more to go!
Hialeah  is done, 3806 more to go!
Fort Lauderdale  is done, 3805 more to go!
Tallahass

...

Chattanooga  is done, 865 more to go!
Clarksville  is done, 864 more to go!
Murfreesboro  is done, 863 more to go!
Jackson  is done, 862 more to go!
An unexpected error occurred processing census for Nashville. Error: City Not Found
Johnson City  is done, 861 more to go!
Franklin  is done, 860 more to go!
Hendersonville  is done, 859 more to go!
Bartlett  is done, 858 more to go!
Kingsport  is done, 857 more to go!
Cleveland  is done, 856 more to go!
Collierville is done, 855 more to go!
Smyrna is done, 854 more to go!
Germantown  is done, 853 more to go!
Brentwood  is done, 852 more to go!
Columbia  is done, 851 more to go!
La Vergne  is done, 850 more to go!
Gallatin  is done, 849 more to go!
Cookeville  is done, 848 more to go!An unexpected error occurred processing census for Collierville. Error: City Not Found

An unexpected error occurred processing census for Smyrna. Error: City Not Found
Oak Ridge  is done, 847 more to go!
Morristown  is done, 846 more to go!
Maryville  is done

Socorro  is done, 682 more to go!
Deer Park  is done, 681 more to go!
Copperas Cove  is done, 680 more to go!
Schertz  is done, 679 more to go!
Cleburne  is done, 678 more to go!
Waxahachie  is done, 677 more to go!
Lake Jackson  is done, 676 more to go!
Weatherford  is done, 675 more to go!
Eagle Pass  is done, 674 more to go!
Southlake  is done, 673 more to go!
Corsicana  is done, 672 more to go!
Farmers Branch  is done, 671 more to go!
Seguin  is done, 670 more to go!
Kyle  is done, 669 more to go!
Paris  is done, 668 more to go!
Greenville  is done, 667 more to go!
Leander  is done, 666 more to go!
Harker Heights  is done, 665 more to go!
San Benito  is done, 664 more to go!
Little Elm  is done, 663 more to go!
University Park  is done, 662 more to go!
Kingsville  is done, 661 more to go!
Colleyville  is done, 660 more to go!
Big Spring  is done, 659 more to go!
Watauga  is done, 658 more to go!
Denison  is done, 657 more to go!
Marshall  is done, 656 more to go!
Benbrook  is done,

Everman  is done, 468 more to go!
Center  is done, 467 more to go!
An unexpected error occurred processing census for Lacy-Lakeview . Error: City Not Found
Cameron  is done, 466 more to go!
Fate  is done, 465 more to go!
Breckenridge  is done, 464 more to go!
Wake Village  is done, 463 more to go!
Rio Bravo  is done, 462 more to go!
Bowie  is done, 461 more to go!
Celina  is done, 460 more to go!
Los Fresnos  is done, 459 more to go!
Diboll  is done, 458 more to go!
Progreso  is done, 457 more to go!
Palmview  is done, 456 more to go!
Atlanta  is done, 455 more to go!
An unexpected error occurred processing census for Rio Bravo . Error: City Not FoundYoakum  is done, 454 more to go!

Luling  is done, 453 more to go!
Giddings  is done, 452 more to go!
Windcrest  is done, 451 more to go!
Sinton  is done, 450 more to go!
Carrizo Springs  is done, 449 more to go!
Brady  is done, 448 more to go!
Rusk  is done, 447 more to go!
Mathis  is done, 446 more to go!
Port Isabel  is done, 445 more t

Marion is done, 293 more to go!
An unexpected error occurred processing census for Ashland. Error: City Not FoundBig Stone Gap is done, 292 more to go!

An unexpected error occurred processing census for Smithfield. Error: City Not Found
An unexpected error occurred processing census for Vinton. Error: City Not FoundEmporia  is done, 291 more to go!

Bridgewater is done, 290 more to go!
An unexpected error occurred processing census for Bedford . Error: City Not Found
Bluefield is done, 289 more to go!
Purcellville is done, 288 more to go!
Virginia is done, 4 more to go!
Seattle  is done, 287 more to go!
Spokane  is done, 286 more to go!
Tacoma  is done, 285 more to go!
An unexpected error occurred processing census for Big Stone Gap. Error: City Not Found
Vancouver  is done, 284 more to go!
An unexpected error occurred processing census for Bridgewater. Error: City Not Found
An unexpected error occurred processing census for Marion. Error: City Not Found
Bellevue  is done, 283 more to

Onalaska  is done, 105 more to go!
Howard is done, 104 more to go!
Menasha  is done, 103 more to go!
Middleton  is done, 102 more to go!
An unexpected error occurred processing census for Germantown. Error: City Not Found
Menomonie  is done, 101 more to go!
An unexpected error occurred processing census for Ashwaubenon. Error: City Not Found
Kaukauna  is done, 100 more to go!
An unexpected error occurred processing census for Howard. Error: City Not FoundBeaver Dam  is done, 99 more to go!

Bellevue is done, 98 more to go!
Allouez is done, 97 more to go!
River Falls  is done, 96 more to go!
Whitewater  is done, 95 more to go!
Oconomowoc  is done, 94 more to go!
An unexpected error occurred processing census for Bellevue. Error: City Not Found
Greendale is done, 93 more to go!
Hartford  is done, 92 more to go!
An unexpected error occurred processing census for Allouez. Error: City Not Found
Whitefish Bay is done, 91 more to go!
Weston is done, 90 more to go!
Shorewood is done, 89 more t

In [15]:
city_df = pd.DataFrame(df_dict)
city_df.head()

,CITY,STATE,PA,POP,UNDER_5,UNDER_18,65_OR_OVER,FEMALE,WHITE,BLACK,...,POVERTY,FIRMS,MEN_FIRM,FEM_FIRM,MINOR_FIRM,NON_MINOR_FIRM,VET_FIRM,NON_VET_FIRM,POP_AREA,AREA
0,Mobile,Alabama,liberal,189572,6.3,21.7,16.0,52.6,44.8,50.6,...,22.0,20118,9703,8644,8808,10250,2275,16639,1402.6,139.11
1,Birmingham,Alabama,liberal,209880,6.5,20.3,14.2,53.1,25.3,70.5,...,27.2,20738,10318,8286,10605,8770,2082,17113,1453.0,146.07
2,Montgomery,Alabama,liberal,198218,6.8,24.0,13.7,53.0,33.3,60.6,...,21.9,18043,8879,7459,8793,8363,2352,14434,1289.5,159.57
3,Tuscaloosa,Alabama,liberal,101113,5.9,18.8,11.7,52.4,51.1,44.1,...,24.2,6802,3374,2530,1953,4375,670,5438,1501.9,60.23
4,Hoover,Alabama,liberal,85108,6.9,25.4,15.6,52.8,72.7,17.4,...,6.7,7982,4402,2471,1371,6147,831,6551,1730.6,47.16


In [16]:
city_df.tail()

,CITY,STATE,PA,POP,UNDER_5,UNDER_18,65_OR_OVER,FEMALE,WHITE,BLACK,...,POVERTY,FIRMS,MEN_FIRM,FEM_FIRM,MINOR_FIRM,NON_MINOR_FIRM,VET_FIRM,NON_VET_FIRM,POP_AREA,AREA
4562,Powell,Wyoming,conservative,6310,8.2,22.9,17.6,51.6,91.6,0.3,...,12.9,945,525,264,39,864,145,736,1487.4,4.25
4563,Casper,Wyoming,conservative,57461,7.4,25.0,13.5,50.5,94.0,1.8,...,9.4,6059,3027,2002,353,5316,721,4824,2056.1,26.90
4564,Torrington,Wyoming,conservative,6701,6.7,21.1,20.2,49.4,93.3,0.1,...,12.9,906,459,325,36,807,117,720,1405.9,4.62
4565,Cody,Wyoming,conservative,9828,6.3,22.1,20.0,52.0,92.3,0.3,...,7.8,1938,951,507,87,1751,285,1448,933.8,10.20
4566,Riverton,Wyoming,conservative,10996,7.0,23.4,16.1,48.8,80.5,1.1,...,13.3,1181,545,396,86,987,48,999,1077.1,9.86


In [20]:
city_df.to_csv('city_stats.csv')